In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 45.4 MB/s eta 0:00:00


In [4]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from shapely.affinity import rotate
import numpy as np
import hashlib

def shapefile_to_binary_vectors(shapefile_path, raster_resolution):
    # Read the shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    all_shape_arrays = []

    for index, row in gdf.iterrows():
        # Get the geometry of the current feature
        geometry = row['geometry']

        # Get the bounding box of the shapefile
        bounding_box = geometry.bounds

        # Calculate the aspect ratio of the original shapefile
        if (bounding_box[3] - bounding_box[1]) > (bounding_box[2] - bounding_box[0]):
          aspect_ratio = (bounding_box[2] - bounding_box[0]) / (bounding_box[3] - bounding_box[1])
          tall = True
        else:
          aspect_ratio = (bounding_box[3] - bounding_box[1]) / (bounding_box[2] - bounding_box[0])
          tall = False
        # tall indicates whether the shape is taller than it is wide or not

        # Adjust placement of shape based on the aspect ratio
        if tall:
          raster_resolutions = (
              int(raster_resolution * aspect_ratio),
              raster_resolution
          )
        else:
          raster_resolutions = (
              raster_resolution,
              int(raster_resolution * aspect_ratio)
          )

        # Create a raster from the shapefile
        with rasterio.Env():
            with rasterio.open(
                'output_raster.tif',
                'w',
                driver='GTiff',
                height=raster_resolutions[1],
                width=raster_resolutions[0],
                count=1,
                dtype=np.uint8,
                crs='EPSG:4326', # Mercator projection
                transform=rasterio.transform.from_bounds(*bounding_box, *raster_resolutions),
            ) as dst:
                # Rasterize the shapefile
                mask = geometry_mask(
                    [geometry],
                    out_shape=(raster_resolutions[1], raster_resolutions[0]),
                    transform=dst.transform,
                    invert=True
                )
                dst.write(mask.astype(np.uint8), 1)

        # Read the raster and convert it to a binary array
        with rasterio.open('output_raster.tif') as src:
            raster_array = src.read(1)
            binary_array = (raster_array > 0).astype(np.uint8)

        shape_array = []

        # Converts array of arrays into a single vector
        if tall:
          for i in range(raster_resolution):
            for j in range(len(binary_array[0])):
              shape_array.append(binary_array[i][j])
            for j in range(raster_resolution - len(binary_array[0])):
              shape_array.append(0)
        else:
          for i in range(len(binary_array)):
            for j in range(raster_resolution):
              shape_array.append(binary_array[i][j])
          for j in range(raster_resolution - len(binary_array)):
            for j in range(raster_resolution):
              shape_array.append(0)

        # Add to list of vectors
        all_shape_arrays.append(shape_array)

    return all_shape_arrays

raster_resolution = 100
shapefile_path2 = 'world_countries.shp'

feature_names = gpd.read_file(shapefile_path2)['CNTRY_NAME'] # Getting names attribute from shapefile

dataset = shapefile_to_binary_vectors(shapefile_path2, raster_resolution)

In [45]:
def minhash_vector(vector, seed):
    # Seed so randomness is always the same
    np.random.seed(seed)

    # Get "hash function" from indices to interval [0,1]
    random_vector = np.random.uniform(0,1,size=len(vector))

    # Start minimum value at 1
    minhash = 1

    # Iterate over the elements of the binary vector.
    for i in range(len(vector)):
      if vector[i] == 1:
        # Calculate the hash value of the element.
        hash_value = random_vector[i]

        # Update the minhash if the hash value is smaller.
        minhash = min(minhash, hash_value)

    # Return the minhash.
    return minhash

def create_tuple_dataset(dataset, r):

    # Create output arrays
    dataset_tuple = np.zeros((len(dataset),r))

    for i in range(r):
      # Generate random seed for hash function
      seed = r_seeds[i]

      # Minhases for each vector in the dataset
      for j in range(len(dataset)):
        dataset_tuple[j][i] = minhash_vector(dataset[j], seed)

    return dataset_tuple

def hash_tuple(tuple_object, m, seed):
    # Convert the tuple object to a string.
    tuple_string = str(tuple_object)

    # Create a hash object with the specified seed.
    hash_object = hashlib.sha256(str(seed).encode('utf-8'))

    # Update the hash object with the tuple string.
    hash_object.update(tuple_string.encode('utf-8'))

    # Get the hash value as a hexadecimal string.
    hash_value = hash_object.hexdigest()

    # Convert the hexadecimal string to an integer.
    hash_value_int = int(hash_value, 16)

    # Return the hash value modulo m.
    return hash_value_int % m + 1

def create_table_dataset(dataset_tuple, t, m):
    # Create output tables
    dataset_table = np.zeros((len(dataset_tuple),t))

    for i in range(t):
      # Generate random seed for hash function
      seed = t_seeds[i]

      # Hashes indexes for each vector in the dataset
      for j in range(len(dataset_tuple)):
        dataset_table[j][i] = hash_tuple(dataset_tuple[j], m, seed)

    return dataset_table

r = 11 # Number of times our minhash funtion will be ran
m = 2 * len(feature_names) # Number of possible indexes to hash to in table
t = 250 # Number of rows in the table

r_seeds = []
t_seeds = []

# Generate random seeds for hash functions
for i in range(r):
    r_seeds.append(np.random.randint(0,1000000))

for i in range(t):
    t_seeds.append(np.random.randint(0,1000000))

# Create tuples for the vector and dataset
dataset_tuple = create_tuple_dataset(dataset, r)

# Create tables for the vector and dataset
dataset_table = create_table_dataset(dataset_tuple, t, m)

In [46]:
def shapefile_to_binary_vector_rotate(shapefile_path, raster_resolution, rotation_angle):
    # Read the shapefile
    gdf = gpd.read_file(shapefile_path)

    # Convert shapefile to polygon through unifying features
    polygon = gdf.union_all()

    # Rotate the polygon
    rotated_polygon = rotate(polygon, rotation_angle)

    # Create a bounds for the rotated polygon
    bounding_box = rotated_polygon.bounds

    # Calculate the aspect ratio of the original shapefile
    if (bounding_box[3] - bounding_box[1]) > (bounding_box[2] - bounding_box[0]):
      aspect_ratio = (bounding_box[2] - bounding_box[0]) / (bounding_box[3] - bounding_box[1])
      tall = True
    else:
      aspect_ratio = (bounding_box[3] - bounding_box[1]) / (bounding_box[2] - bounding_box[0])
      tall = False
    # tall indicates whether the shape is taller than it is wide or not

    # Adjust placement of shape based on the aspect ratio
    if tall:
      raster_resolutions = (
          int(raster_resolution * aspect_ratio),
          raster_resolution
      )
    else:
      raster_resolutions = (
          raster_resolution,
          int(raster_resolution * aspect_ratio)
      )

    # Create a raster from the rotated polygon
    with rasterio.Env():
        with rasterio.open(
            'output_raster.tif',
            'w',
            driver='GTiff',
            height=raster_resolutions[1],
            width=raster_resolutions[0],
            count=1,
            dtype=np.uint8,
            crs="EPSG:4326", # Mercator projection
            transform=rasterio.transform.from_bounds(*bounding_box, *raster_resolutions),
        ) as dst:
            mask = geometry_mask(
                [rotated_polygon],
                out_shape=(raster_resolutions[1], raster_resolutions[0]),
                transform=dst.transform,
                invert=True
            )
            dst.write(mask.astype(np.uint8), 1)

    # Read the raster and convert it to a binary array
    with rasterio.open('output_raster.tif') as src:
        raster_array = src.read(1)
        binary_array = (raster_array > 0).astype(np.uint8)

    shape_array = []

    # Converts array of arrays into a single vector
    if tall:
      for i in range(raster_resolution):
        for j in range(len(binary_array[0])):
          shape_array.append(binary_array[i][j])
        for j in range(raster_resolution - len(binary_array[0])):
          shape_array.append(0)
    else:
      for i in range(len(binary_array)):
        for j in range(raster_resolution):
          shape_array.append(binary_array[i][j])
      for j in range(raster_resolution - len(binary_array)):
        for j in range(raster_resolution):
          shape_array.append(0)

    return shape_array

def create_tuple_vector(vector, r):

    # Create output arrays
    vector_tuple = np.zeros(r)

    for i in range(r):
      # Generate random seed for hash function
      seed = r_seeds[i]

      # Minhashes for sample vector
      vector_tuple[i] = minhash_vector(vector, seed)

    return vector_tuple

def create_table_vector(vector_tuple, t, m):
    # Create output tables
    vector_table = np.zeros(t)

    for i in range(t):
      # Generate random seed for hash function
      seed = t_seeds[i]

      # Hashes indexes for sample vector
      vector_table[i] = hash_tuple(vector_tuple, m, seed)

    return vector_table

def check_table(vector_table, dataset_table):
    # Create list for indexes to be appended on to
    check_list = []

    # Iterates through table to find matches in the sample vector to the dataset
    for i in range(len(vector_table)):
      for j in range(len(dataset_table)):
        if vector_table[i] == dataset_table[j][i]:
          check_list.append(j)

    return list(set(check_list))

def jaccards(vector1, vector2):
    intersection = 0
    union = 0

    # For each 1 or 0 in the vectors, the intersection or union value will increase
    for i in range(len(vector1)):
      if vector1[i] == 1 and vector2[i] == 1:
        intersection += 1
      if vector1[i] == 1 or vector2[i] == 1:
        union += 1

    # Returns similairty value from 0 to 1. 1 is most similar.
    return intersection / union

def max_similarity(vector, dataset, check_list):
    # Set initial values for the maximum similarity and the maximum index at that similarity
    max_similarity = 0
    max_index = 0

    # Find similarity for each index in the check list, store only maximum
    for i in range(len(check_list)):
      sim = jaccards(vector, dataset[check_list[i]])
      if max_similarity < sim:
        max_similarity = sim
        max_index = check_list[i]

    return max_similarity, max_index

# Example usage
shapefile_path1 = 'VT_counties.shp'

total_max_sim = 0

total_max_index = 0

max_degree = 0

# Repeat for multiple rotations
for i in range(72):
  # get a binary vector from shapefile
  vector = shapefile_to_binary_vector_rotate(shapefile_path1, raster_resolution, i * 5)

  # Create tuples for the vector and dataset
  vector_tuple = create_tuple_vector(vector, r)

  # Create tables for the vector and dataset
  vector_table = create_table_vector(vector_tuple, t, m)

  # Generate a list of which shapes in the dataset to check
  check_list = check_table(vector_table, dataset_table)

  print("Checking",len(check_list),"countries for your shape rotated at", i * 5, "degrees.")

  # Find maximum similarity for all checks
  max_sim, max_index = max_similarity(vector, dataset, check_list)
  if max_sim > total_max_sim:
    total_max_sim = max_sim
    total_max_index = max_index
    max_degree = i * 5

max_country = feature_names[total_max_index]

print('The country most similar to your shape is',max_country,'with a similarity score of',total_max_sim, 'rotated at', max_degree, 'degrees.')

Checking 92 countries for your shape rotated at 0 degrees.
Checking 93 countries for your shape rotated at 5 degrees.
Checking 105 countries for your shape rotated at 10 degrees.
Checking 99 countries for your shape rotated at 15 degrees.
Checking 104 countries for your shape rotated at 20 degrees.
Checking 86 countries for your shape rotated at 25 degrees.
Checking 86 countries for your shape rotated at 30 degrees.
Checking 87 countries for your shape rotated at 35 degrees.
Checking 102 countries for your shape rotated at 40 degrees.
Checking 84 countries for your shape rotated at 45 degrees.
Checking 110 countries for your shape rotated at 50 degrees.
Checking 88 countries for your shape rotated at 55 degrees.
Checking 105 countries for your shape rotated at 60 degrees.
Checking 110 countries for your shape rotated at 65 degrees.
Checking 88 countries for your shape rotated at 70 degrees.
Checking 102 countries for your shape rotated at 75 degrees.
Checking 89 countries for your shap